Black-Scholes model

In [1]:
from scipy.stats import norm
from scipy.stats import t
import numpy as np
import math

S = 100
K = 110
T = 0.4
r = 0.05
q = 0.02
sigma = 0.5


def black_scholes(S, K, T, r, q, sigma, option='call'):
    d1 = (math.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = (math.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    
    if option == 'call':
        return S * math.exp(-q*T) * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    if option == 'put':
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1) * math.exp(-q*T)
    
call_price = black_scholes(S,K,T,r,q,sigma,option="call")
put_price = black_scholes(S,K,T,r,q,sigma,option="put")

print(f"Call Price: {call_price:.4f}")
print(f"Put Price: {put_price:.4f}")

Call Price: 9.1399
Put Price: 17.7586


Monte-Carlo's simulation

In [2]:
import numpy as np
import math
from scipy.stats import norm

n_simulations = 10000
n_steps = 20

def monte_carlo(S, K, T, r, q, sigma, n_simulations, n_steps, option = 'call'):
    if option == "call":
        call_prices = []
        for _ in range(n_steps):
            rn = np.random.normal(0, 1, n_simulations)
            ST = S * np.exp((r - q - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * rn)
            payoff = np.exp(-r*T) * np.maximum((ST - K), 0)
            average = np.mean(payoff)
            call_prices.append(average)            
        mean = np.mean(call_prices)
        std = np.std(call_prices)
        upper = np.round(mean + 2 * std,4)
        lower = np.round(mean - 2 * std,4) 
        CI_call = [lower,upper]
        return mean,std,CI_call

    if option == "put":
        put_prices = []
        for _ in range(n_steps):
            rn = np.random.normal(0, 1, n_simulations)
            ST = S * np.exp((r - q - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * rn)
            payoff = np.exp(-r*T) * np.maximum((K-ST), 0)
            average = np.mean(payoff)
            put_prices.append(average)            
        mean = np.mean(put_prices)
        std = np.std(put_prices)
        upper = np.round(mean + 2 * std,4)
        lower = np.round(mean - 2 * std,4)      
        CI_put = [lower, upper]
        return mean,std,CI_put 

mean_call,std_call,CI_call = monte_carlo(S,K,T,r,q,sigma,n_simulations, n_steps, option = 'call')
mean_put,std_put,CI_put = monte_carlo(S,K,T,r,q,sigma,n_simulations, n_steps, option = 'put')
print(f"mean of call: {mean_call:.4f}", f"std of call: {std_call:.4f}", f"CI of call: {CI_call}")
print(f"mean of put: {mean_put:.4f}", f"std of call: {std_put:.4f}", f"CI of call: {CI_put}")

mean of call: 9.1259 std of call: 0.2283 CI of call: [8.6693, 9.5824]
mean of put: 17.7443 std of call: 0.1434 CI of call: [17.4576, 18.031]


In [3]:
N = 100

def binomial_tree(S=100, K=102, r=0.05, q=0.02, sigma=0.2, T=1, N=100, option = "European Call"):
    dt = T/N
    u = np.exp( sigma * np.sqrt(dt))
    d = np.exp( - sigma * np.sqrt(dt))
    p = (np.exp((r - q) * dt) - d) / (u - d)  # risk-neutral probability

    #建立股價空間
    price_tree = np.zeros((N+1,N+1))
    price_tree[0,0] = S

    #輸入預期股價
    for i in range(1,N+1):
        price_tree[i,0] = price_tree[i-1,0] * u
        for j in range(1,i+1):
            price_tree[i,j] = price_tree[i-1,j-1] * d

    if option == "European Call":
        # Option valuation
        option_tree = np.zeros((N+1,N+1))
        option_tree[N] = np.maximum(price_tree[N] - K, 0)
        #backward induction
        for i in range(N-1,-1,-1):
            for j in range(0,i+1):
                option_tree[i,j] = np.exp(-r * dt) * (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j+1])
        return round(option_tree[0,0],4)
    
    if option == "European Put":
        # Option valuation
        option_tree = np.zeros((N+1,N+1))
        option_tree[N] = np.maximum(K-price_tree[N], 0)
        #backward induction
        for i in range(N-1,-1,-1):
            for j in range(0,i+1):
                option_tree[i,j] = np.exp(-r * dt) * (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j+1])
        return round(option_tree[0,0],4)
    

    if option == "American Call":
        # Option valuation
        option_tree = np.zeros((N+1,N+1))
        option_tree[N] = np.maximum(price_tree[N]-K, 0)
        #backward induction
        for i in range(N-1,-1,-1):
            for j in range(0,i+1):
                hold = np.exp(-r * dt) * (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j+1])
                exercise = price_tree[i,j]-K
                option_tree[i,j] = np.maximum(hold,exercise)
        return round(option_tree[0,0],4)
    
    if option == "American Put":
        # Option valuation
        option_tree = np.zeros((N+1,N+1))
        option_tree[N] = np.maximum(K-price_tree[N], 0)
        #backward induction
        for i in range(N-1,-1,-1):
            for j in range(0,i+1):
                hold = np.exp(-r * dt) * (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j+1])
                exercise = K-price_tree[i,j]
                option_tree[i,j] = np.maximum(hold,exercise)
        return round(option_tree[0,0],4)
    
E_C = binomial_tree(S=S,K=K,r=r,q=q,sigma=sigma,T=T,N=N,option = "European Call")
E_P = binomial_tree(S=S,K=K,r=r,q=q,sigma=sigma,T=T,N=N,option = "European Put")
A_C = binomial_tree(S=S,K=K,r=r,q=q,sigma=sigma,T=T,N=N,option = "American Call")
A_P = binomial_tree(S=S,K=K,r=r,q=q,sigma=sigma,T=T,N=N,option = "American Put")

print(f"European Call: {E_C}")
print(f"European Put: {E_P}")    
print(f"American Call: {A_C}")    
print(f"American Put: {A_P}")    

European Call: 9.1696
European Put: 17.7882
American Call: 9.1696
American Put: 17.9983
